Data were moved to cd ../Drives/HDD03_06T_SDE/anna/SNP_Pipeline_Data/Data

# Mapping the sequencing data to the reference Genome

Tools: Minimap2, samtools sort, samtools index

In [1]:
cd ~

In [2]:
cd ../Drives/HDD03_06T_SDE/anna/SNP_Pipeline_Data/Data/RefGenome/

Map the data to the reference genome with Minimap2

In [2]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do minimap2 -a $i.fasta ../Sequencing/RawReads.fastq > ../Alignments/Raw/$i.sam; done

[M::mm_idx_gen::0.126*1.01] collected minimizers
[M::mm_idx_gen::0.151*1.32] sorted minimizers
[M::main::0.151*1.32] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.165*1.30] mid_occ = 10
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.177*1.28] distinct minimizers: 556909 (94.43% are singletons); average occurrences: 1.068; average spacing: 5.362; total length: 3188530
[M::worker_pipeline::38.105*3.00] mapped 3333334 sequences
[M::worker_pipeline::70.523*3.12] mapped 3333334 sequences
[M::worker_pipeline::98.781*3.17] mapped 3333334 sequences
[M::worker_pipeline::129.577*3.19] mapped 3333334 sequences
[M::worker_pipeline::158.924*3.19] mapped 3333334 sequences
[M::worker_pipeline::169.511*3.15] mapped 1440442 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: minimap2 -a Chr1.fasta ../Sequencing/RawReads.fastq
[M::main] Real time: 170.004 sec; CPU: 533.699 sec; Peak RSS: 2.985 GB
[M::mm_idx_gen::0.041*1.05] collected minimize

Sort the .sam files with samtools (Covert to .bam)

In [19]:
cd ~

In [20]:
cd ../Drives/HDD03_06T_SDE/anna/SNP_Pipeline_Data/Data/Alignments/Raw

In [4]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools sort -o $i.bam $i.sam; done

[bam_sort_core] merging from 8 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...


Remove the .sam files to reduce memory usage

In [5]:
for i in *.sam; do rm $i; done

Indexing the .bam files with samtools

In [6]:
for i in *.bam; do samtools index $i; done

## Filtering the maps (MAPQ >=30)

In [7]:
cd ~/SnP_Pipeline_full/Data/Alignments

In [8]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools view -q 30 -b Raw/$i.bam > Filtered/$i.q30.bam; done

# Mapping with bowtie2

In [1]:
cd /home/Drives/HDD03_06T_SDE/anna/SNP_Pipeline_Data/Data/RefGenome

In [2]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do bowtie2-build -f $i.fasta indices/$i; done

Settings:
  Output files: "indices/Chr1.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  Chr1.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 797132
Using parameters --bmax 597849 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 597849 --dcv 1024
Constructing suffix-array eleme

In [3]:
cd ./indices/

In [4]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do bowtie2 -x $i -f -U /home/Drives/HDD03_06T_SDE/anna/SNP_Pipeline_Data/Data/Sequencing/RawReads.fasta -S /home/Drives/HDD03_06T_SDE/anna/SNP_Pipeline_Data/Data/Alignments/bowtie2_$i.sam --threads 10; done

18107112 reads; of these:
  18107112 (100.00%) were unpaired; of these:
    4871575 (26.90%) aligned 0 times
    13059857 (72.13%) aligned exactly 1 time
    175680 (0.97%) aligned >1 times
73.10% overall alignment rate
18107112 reads; of these:
  18107112 (100.00%) were unpaired; of these:
    14272320 (78.82%) aligned 0 times
    3676399 (20.30%) aligned exactly 1 time
    158393 (0.87%) aligned >1 times
21.18% overall alignment rate
18107112 reads; of these:
  18107112 (100.00%) were unpaired; of these:
    17735694 (97.95%) aligned 0 times
    362740 (2.00%) aligned exactly 1 time
    8678 (0.05%) aligned >1 times
2.05% overall alignment rate
18107112 reads; of these:
  18107112 (100.00%) were unpaired; of these:
    17931149 (99.03%) aligned 0 times
    175963 (0.97%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
0.97% overall alignment rate
18107112 reads; of these:
  18107112 (100.00%) were unpaired; of these:
    17917128 (98.95%) aligned 0 times
    164360 (0.91%) alig

In [5]:
cd /home/Drives/HDD03_06T_SDE/anna/SNP_Pipeline_Data/Data/Alignments

In [8]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools sort -o bowtie2_$i.bam bowtie2_$i.sam; done

[bam_sort_core] merging from 8 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...
[bam_sort_core] merging from 7 files and 1 in-memory blocks...


In [9]:
for i in *.sam; do rm $i; done

In [10]:
cp bowtie2_* bowtie/Raw; rm bowtie2_*; cd bowtie/Raw

In [11]:
for i in *.bam; do samtools index $i; done

In [14]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools view -q 30 -b bowtie2_$i.bam > ../Filtered/$i.q30.bam; done

Get some statistics

In [15]:
cd ../Filtered; for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools depth  $i.q30.bam > $i.tab; done

# Variant Calling with BCFTools

## With filtered mapping (minimap2)

In [14]:
cd ~/SnP_Pipeline_full/Data/RefGenome

In [15]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do bcftools mpileup -Ou -f $i.fasta ../Alignments/Filtered/$i.q30.bam  | bcftools call -mv --ploidy 1 -Ob -o ../SNPCalls/BCFTools/Filtered/$i.bcf; done

[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250


In [16]:
cd ../SNPCalls/BCFTools/Filtered

In [17]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do bcftools convert $i.bcf > $i.vcf; done

In [18]:
for i in *.bcf; do rm $i; done

# Variant Calling with Freebayes

## With unfiltered Mapping

In [19]:
cd ~/SnP_Pipeline_full/Data/RefGenome

Call the variants (SNPs and Indels) from the .bam file against the reference sequence

In [20]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do freebayes -f $i.fasta ../Alignments/Raw/$i.bam --ploidy 1 > ../SNPCalls/Freebayes/Raw/$i.vcf; done

## With filtered Mapping

In [21]:
cd ~/SnP_Pipeline_full/Data/RefGenome

Call the variants (SNPs and Indels) from the .bam file against the reference sequence

In [22]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do freebayes -f $i.fasta ../Alignments/Filtered/$i.q30.bam --ploidy 1 > ../SNPCalls/Freebayes/Filtered/$i.vcf; done

# Variant Calling with LoFreq

In [23]:
cd ~/SnP_Pipeline_full/Data/RefGenome

In [24]:
conda activate lofreq

## With unfiltered Mapping

In [27]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do lofreq indelqual --dindel --ref $i.fasta --out ~/SnP_Pipeline_full/Data/Alignments/IndelQual/$i.indelqual.bam ~/SnP_Pipeline_full/Data/Alignments/Raw/$i.bam; done

In [29]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do lofreq call -f ~/SnP_Pipeline_full/Data/RefGenome/$i.fasta --call-indels -o ~/SnP_Pipeline_full/Data/SNPCalls/Lofreq/Raw/$i.vcf ~/SnP_Pipeline_full/Data/Alignments/IndelQual/$i.indelqual.bam; done 

WARNING(lofreq_call.c|main_call): 3 indel calls (before filtering) were made without indel alignment-quality! Did you forget to indel alignment-quality to your bam-file?
Number of substitution tests performed: 5378520
Number of indel tests performed: 39363
Number of substitution tests performed: 1566852
Number of indel tests performed: 12436
Number of substitution tests performed: 169833
Number of indel tests performed: 1244
Number of substitution tests performed: 89514
Number of indel tests performed: 697
Number of substitution tests performed: 89250
Number of indel tests performed: 611
WARNING(lofreq_call.c|main_call): 1 indel calls (before filtering) were made without indel alignment-quality! Did you forget to indel alignment-quality to your bam-file?
Number of substitution tests performed: 90213
Number of indel tests performed: 797


In [30]:
cd ../Alignments/IndelQual/

In [31]:
rm *.indelqual.bam

## With filtered Mapping

In [32]:
cd ~/SnP_Pipeline_full/Data/RefGenome

In [33]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do lofreq indelqual --dindel --ref $i.fasta --out ~/SnP_Pipeline_full/Data/Alignments/IndelQual/$i.indelqual.bam ~/SnP_Pipeline_full/Data/Alignments/Filtered/$i.q30.bam; done

In [34]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do lofreq call -f ~/SnP_Pipeline_full/Data/RefGenome/$i.fasta --call-indels -o ~/SnP_Pipeline_full/Data/SNPCalls/Lofreq/Filtered/$i.vcf ~/SnP_Pipeline_full/Data/Alignments/IndelQual/$i.indelqual.bam; done 

WARNING(lofreq_call.c|main_call): 3 indel calls (before filtering) were made without indel alignment-quality! Did you forget to indel alignment-quality to your bam-file?
Number of substitution tests performed: 5291154
Number of indel tests performed: 34485
WARNING(lofreq_call.c|main_call): 1 indel calls (before filtering) were made without indel alignment-quality! Did you forget to indel alignment-quality to your bam-file?
Number of substitution tests performed: 1515621
Number of indel tests performed: 10392
Number of substitution tests performed: 166440
Number of indel tests performed: 1110
Number of substitution tests performed: 86928
Number of indel tests performed: 566
Number of substitution tests performed: 80055
Number of indel tests performed: 491
WARNING(lofreq_call.c|main_call): 1 indel calls (before filtering) were made without indel alignment-quality! Did you forget to indel alignment-quality to your bam-file?
Number of substitution tests performed: 36552
Number of indel tes